In [ ]:

import os
import h5py
import ast
import json
import time

import pandas as pd
import hdbscan
import numpy as np
from numpy import linalg

# ML imports
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import SpectralClustering
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph

from sklearn import svm
from sklearn.datasets import make_moons, make_blobs
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

from sklearn.metrics.pairwise import pairwise_distances
import nestle

# Sklearn
import sklearn
from sklearn import datasets
from sklearn.manifold import TSNE
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn import datasets
from sklearn import svm
from sklearn.semi_supervised import label_propagation
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.semi_supervised import label_propagation
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import davies_bouldin_score, silhouette_score, calinski_harabaz_score


# Scipy
from scipy import stats
from scipy.stats import multivariate_normal
import scipy as sp
from scipy.spatial.distance import cdist


# Torch imports
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim


# Skorch imports
from skorch import NeuralNetClassifier
from skorch.helper import predefined_split
from skorch.callbacks import Freezer
from skorch import NeuralNetClassifier

# image library imports
from PIL import Image
import mahotas
import imageio
import ImageFeatures

# skimage imports
import skimage
from skimage.filters import gaussian
from skimage.filters.rank import bottomhat
from skimage.feature import hog
from skimage import color
from skimage.color import hsv2rgb
from skimage.future import graph as rag
from skimage.color import label2rgb
from skimage.color import rgb2hsv
from skimage import segmentation as seg
from skimage import exposure
import skimage.io as io
from skimage.color import rgb2lab
from scipy import ndimage
from skimage.util import img_as_float, img_as_ubyte
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from skimage.feature import canny
from skimage.morphology import skeletonize, medial_axis
from skimage.morphology import erosion, dilation
from skimage.morphology import disk
from skimage.segmentation import felzenszwalb


# Plotting imports
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
from matplotlib import animation
from matplotlib import cm
from colorspacious import cspace_converter
from collections import OrderedDict
import matplotlib.colors as colors
import seaborn as sns

#import plotly.plotly as py
from numpy.random import permutation

# Project-specific imports

from ClusterUtility import (ocv_to_sk, sk_to_ocv,
                            plot_img_and_hist, show_images, get_summary_stats,
                            make_branchpoints, make_endpoints, get_clean_label_im,
                            parse_tuple, randrange, plot_ellipsoid_3d,
                            plot_2d_data, plot_3d_data, plot_3d_data_in_2d,plot_4d_data)



from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import cm
from pylab import meshgrid,linspace,zeros,dot,norm,cross,vstack,array,matrix,sqrt

import PIL
import PIL.Image
from skimage import img_as_float, img_as_ubyte

import numpy as np
from scipy.spatial import Delaunay
from scipy.interpolate import interpn
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook
import numpy as np
import ipyvolume
import ipyvolume as ipv
import ipyvolume.pylab as p3

#
# Plotting parameters
#

# Cmaps worth trying

cmaps = OrderedDict()

cmaps = ['viridis', 'terrain', 'Wistia', 'inferno', 'magma', 'cividis',
            'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
            'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn',
            'spring', 'summer', 'autumn', 'winter', 'cool', 
            'hot', 'afmhot', 'gist_heat', 'copper', 'Pastel1', 'tab10', 'tab20']

plt.rcParams.update({'font.size': 12})

matplotlib.rcParams['contour.negative_linestyle'] = 'solid'

In [ ]:
fpath = "/dls/science/groups/das/zooniverse/virus_factory/data/"
fname = "combined_zooniverse_data.h5"
#fname = "virus_factory_source_data.h5"

hdf5_data = h5py.File(os.path.join(fpath, fname), 'r')

# In[]
hdf5_data.keys()
ds = hdf5_data.get('dataset')
wf1 = ds.get('workflow_1')  # <HDF5 dataset "workflow1_tv_denoised_padded": shape (165, 2112, 2160), type "<f4">
wf2 = ds.get('workflow_2')  # <HDF5 dataset "workflow2_gauss_filtered_padded": shape (165, 2112, 2160), type "<f4">
# In[]

ds_export = hdf5_data.get('data_export')
wf1 = ds.get('workflow_1')  # <HDF5 dataset "workflow1_tv_denoised_padded": shape (165, 2112, 2160), type "<f4">
wf1_wrangled = ds_export.get('workflow1_wrangled_export')

vol_shape_x = wf1[0].shape[0]
vol_shape_y = wf1[0].shape[1]
vol_shape_z = len(wf1)
wf1_roi = wf1_wrangled['parent_data_roi']
# In[]
win = hdf5_data.get('windows')
print(win.keys())
#<KeysViewHDF5 ['workflow_1_images', 'workflow_1_regionref']>
wf1_im = win.get('workflow_1_images')  #  <HDF5 dataset "workflow_1_images": shape (5865, 264, 240), type "<f4">
wf1_regref = win.get('workflow_1_regionref')  # <HDF5 dataset "workflow_1_regionref": shape (5865,), type "|O">
# In[]

#
# TODO: better HDF manipulation
# 


def walk_func(name, obj):
   print(name, obj)
   return None


hdf5_data.visititems(walk_func)


"""
data_export <HDF5 group "/data_export" (2 members)>
dataset <HDF5 group "/dataset" (2 members)>
windows <HDF5 group "/windows" (2 members)>
windows/workflow_1_images <HDF5 dataset "workflow_1_images": shape (5865, 264, 240), type "<f4">
windows/workflow_1_regionref <HDF5 dataset "workflow_1_regionref": shape (5865,), type "|O">
"""

#region references
#name = h5py.h5r.get_name(ref, file.id)

%pylab inline 

for i in range(545,550):
    plt.figure()
    plt.imshow(wf1_im[i*10],cmap='gray')


# In[]
def parse_tuple(string):
    try:
        s = ast.literal_eval(str(string))
        if type(s) == tuple:
            return s
        return
    except:
        return
    
session_extract = []

def extract_session_roi(roi_data, range_start=0, range_end=50, plot_slices=False, debug_verbose=False):

    click_data = []  # list of per-session data
    
    for idx in range(range_start, range_end):
        
        print("Extracting roi for parent_data_roi index: {}".format(idx))
    
          
        roi_str = roi_data['parent_data_roi'][idx]
                  
        roi =  parse_tuple(roi_str)
        
        sliceno, xstart, ystart, xend, yend = roi
        
        session_anno = parse_tuple(roi_data['roi_coord_tuples'][idx])

        classification_id, roi_str2, clicks_list = session_anno

        session_extract.append((classification_id, roi_str2))        
        
        clicks_arr = np.array(clicks_list)

        if debug_verbose:
            print(roi_str)
            
            print("Session data: {}".format(session_anno))
    
    
            print("Roi strings: {}\n {}", roi_str, roi_str2)
    
            print("Clicks array: {}".format(clicks_arr))        
        # extract per-session data and create list of session data 
        session = []
        
        
        xs=[] # just for visualisation below
        ys=[]
        
        for click_num in range(clicks_arr.shape[0]):
            x = xstart + clicks_arr[click_num,0]
            y = ystart + clicks_arr[click_num,1]
            
            session.append((sliceno, x,y))
            xs.append(x)
            ys.append(y)
        
        click_data.append(session)
        
        # For viz and debug
        if plot_slices:
            imstack = wf1
            
            droi = imstack[sliceno,:,:].copy()
            droi = np.fliplr(droi)
           
            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.imshow(droi[xstart:xend,ystart:yend])
            
            plt.scatter(xs,ys, c='red')
            
    return click_data



def get_window(image_volume, sliceno, xstart, ystart, xend, yend):
        return image_volume[sliceno, xstart:xend, ystart:yend]
    
#click_data = extract_session_roi(wf1_wrangled, range_start=0, range_end=14384, debug_verbose=False)
    
click_data = extract_session_roi(wf1_wrangled, range_start=0, range_end=2000, debug_verbose=False)




In [ ]:
samp_idx = 13750
samp_idx = 6800
samp_idx = 450

samp_idx = 13450

for samp_idx in range(samp_idx,samp_idx+10):
    session_anno = parse_tuple(wf1_wrangled['roi_coord_tuples'][samp_idx])
    classification_id, roi_str2, clicks_list = session_anno
    clicks_arr = np.array(clicks_list).astype(np.uint16)
    sliceno, xstart, ystart, xend, yend = parse_tuple(wf1_roi[samp_idx])
    
    print(classification_id)
    
    xoffset = 0
    yoffset = 0
    #yoffset  = 25
    #xoffset = -25
    
    print("Slice no: {}, xstart: {}, ystart: {}, xend: {}, yend: {}".format(sliceno, xstart, ystart, xend, yend))
    img = get_window(wf1, sliceno, xstart+xoffset, ystart+yoffset, xend+xoffset, yend+yoffset)
    
    print("Image shape: {}".format(img.shape))
    xs= clicks_arr[:,0]
    ys = clicks_arr[:,1]
    
    fig, ax = plt.subplots()
    
    #plt.gca().invert_yaxis()
    plt.imshow(img.T, origin='lower')
    plt.title("Samp_idx: {}".format(samp_idx))
    xmin, xmax = 0,240
    ymin, ymax = 0,264
    
    ax.set_xlim([xmin,xmax])
    ax.set_ylim([ymin,ymax])
    plt.scatter(ys, xs,   c='red')
    
    
    print("Max x click, max y-click: {} {}".format(np.max(xs), np.max(ys)))



In [ ]:

#
# Cropping image volume 
# 

def plot_points_on_slice(sliceno):
    
    #crop_roi =  (84, 500, 1000, 1000, 1500) # nice big patch of viruses
    
    #crop_roi =  (84, 600, 1000, 800, 1200)
    #crop_roi =  (74, 600, 1000, 800, 1200)
    #crop_roi =  (64, 300, 300, 1000, 1000)
    #crop_roi = (0,300,300,1200,1200)
    crop_roi = (sliceno,0,0,vol_shape_x, vol_shape_y)
    cropped_stack = []
    slicestart, sliceend = crop_roi[0]-10,crop_roi[0]+10

    scaled_cropped_stack = []
    
    
    for sliceno in range(slicestart,sliceend):
        _, xstart, ystart, xend, yend = crop_roi
        print("Slice no.: {}".format(sliceno))        
        win_img = get_window(wf1, sliceno, xstart,ystart,xend,yend)
            
        cropped_stack.append(win_img)
        #scaled_cropped_stack.append(skimage.transform.rescale(win_img, 0.5))
    
        
    cropped_stack = np.array(cropped_stack)
        
    #
    # basic filtering of click coords for crop
    #
    
    accum = 0
    lengths = []
    click_coords = []
    
    _, xstart, ystart, xend, yend = crop_roi
    
    
    for i in range(len(click_data)):
        length = len(click_data[i])
        accum = accum + length
        lengths.append(length)
        for c in click_data[i]:
            print(c)
            if c[1] > xstart and c[1] < xend and c[2] > ystart and c[2] < yend and c[0] > slicestart and c[0] < sliceend:
                click_coords.append([c[1], c[2], c[0]])
    #coords_arranged = []
    #
    #for coords,c in zip(click_data, colors):
    #    for coord in coords:
    #        coords_arranged.append([coord[1], coord[2], coord[0]])
    
    
    # select clicks that are within a slice range
    
    
    
    click_data_arr = np.concatenate(click_data)
    print("Max click coords, (x,y): ({},{})".format(np.max(click_data_arr[:,1]), np.max(click_data_arr[:,2])))
    print("Volume shape x, y: {}, {}".format(vol_shape_x, vol_shape_y))
    
    x_coords_range = np.max(click_data_arr[:,1]) -  np.min(click_data_arr[:,1])
    y_coords_range = np.max(click_data_arr[:,2]) -  np.min(click_data_arr[:,2])
    
    print("x coords range: {}".format(x_coords_range))
    print("y coords range: {}".format(y_coords_range))
    
    click_data_arr[:,1] = click_data_arr[:,1] 
    click_data_arr[:,2] = click_data_arr[:,2] 
    
    scale_factor_x = vol_shape_x / y_coords_range
    scale_factor_y = vol_shape_y / x_coords_range
    
    print(scale_factor_x, scale_factor_y)

    #cropped_clicks = click_data_arr[np.where(np.logical_and(click_data_arr[:,0]>=56, click_data_arr[:,0]<=62))]   
    sel_clicks = click_data_arr[np.where(
            np.logical_and(click_data_arr[:,0]>=sliceno-15, click_data_arr[:,0]<=sliceno+15))]            
    
    print("Selected {} clicks".format(sel_clicks.shape[0]))
    
    subvol_idx =20
    vol_idx = subvol_idx + slicestart
    sel_coords = []
    sel_clicks


    #for i in range(len(click_coords)):
    #    if click_coords[i][2] == vol_idx:
    #        xs.append(click_coords[i][0]-xstart)        
    #        ys.append(click_coords[i][1]-ystart)
    """
    The padding looks like its nonuninform in x and y
    """
        
    scale_factor_x, scale_factor_y = 1,1
    
    offset_x, offset_y = 0,0
    
    xs = offset_x + sel_clicks[:,1] * scale_factor_x
    ys = offset_y + sel_clicks[:,2] * scale_factor_y
    
    #proc_img = cropped_stack[subvol_idx]
    _, xstart, ystart, xend, yend = crop_roi
    proc_img = get_window(wf1, vol_idx, xstart,ystart,xend,yend)
    
    print("Image shape: {}".format(proc_img.shape))    
            
    fig, ax = plt.subplots(figsize=(6,6))

    plt.imshow(proc_img.T, cmap='gray', origin='lower')    
    plt.title('2d slice no: {} with {} clicks plotted'.format(vol_idx, sel_clicks.shape[0]))
    
    #plt.figure()
    ax.set_xlim([0,xend])
    ax.set_ylim([0,yend])
    plt.gca().set_aspect('equal', adjustable='box')
    plt.scatter(ys, xs,  c='cyan', alpha=0.4)
    #


for i in range(10,150, 10):   
    plot_points_on_slice(i)

In [ ]:
sliceno=80
crop_roi = (sliceno,0,0,vol_shape_x, vol_shape_y)
cropped_stack = []
slicestart, sliceend = crop_roi[0]-40,crop_roi[0]+40

scaled_cropped_stack = []


for sliceno in range(slicestart,sliceend):
    _, xstart, ystart, xend, yend = crop_roi
    print("Slice no.: {}".format(sliceno))        
    win_img = get_window(wf1, sliceno, xstart,ystart,xend,yend)

    cropped_stack.append(win_img)
    #scaled_cropped_stack.append(skimage.transform.rescale(win_img, 0.5))


cropped_stack = np.array(cropped_stack)

#
# basic filtering of click coords for crop
#

accum = 0
lengths = []
click_coords = []

_, xstart, ystart, xend, yend = crop_roi


for i in range(len(click_data)):
    length = len(click_data[i])
    accum = accum + length
    lengths.append(length)
    for c in click_data[i]:
        print(c)
        if c[1] > xstart and c[1] < xend and c[2] > ystart and c[2] < yend and c[0] > slicestart and c[0] < sliceend:
            click_coords.append([c[1], c[2], c[0]])

In [ ]:
click_data_arr = np.concatenate(click_data)
print("Max click coords, (x,y): ({},{})".format(np.max(click_data_arr[:,1]), np.max(click_data_arr[:,2])))
print("Volume shape x, y: {}, {}".format(vol_shape_x, vol_shape_y))

x_coords_range = np.max(click_data_arr[:,1]) -  np.min(click_data_arr[:,1])
y_coords_range = np.max(click_data_arr[:,2]) -  np.min(click_data_arr[:,2])

print("x coords range: {}".format(x_coords_range))
print("y coords range: {}".format(y_coords_range))

click_data_arr[:,1] = click_data_arr[:,1] 
click_data_arr[:,2] = click_data_arr[:,2] 

scale_factor_x = vol_shape_x / y_coords_range
scale_factor_y = vol_shape_y / x_coords_range

print(scale_factor_x, scale_factor_y)

#cropped_clicks = click_data_arr[np.where(np.logical_and(click_data_arr[:,0]>=56, click_data_arr[:,0]<=62))]   
sel_clicks = click_data_arr[np.where(
        np.logical_and(click_data_arr[:,0]>10, click_data_arr[:,0]<=160))]            

print("Selected {} clicks".format(sel_clicks.shape[0]))

subvol_idx =0
vol_idx = subvol_idx + slicestart
sel_coords = []

scale_factor_x, scale_factor_y = 1,1
offset_x, offset_y = 0,0

xs = offset_x + sel_clicks[:,1] * scale_factor_x
ys = offset_y + sel_clicks[:,2] * scale_factor_y


In [ ]:
#print(np.min(sel_clicks[:,0]), np.max(sel_clicks[:,0]))
#sel_clicks.shape



#ipyvolume.volshow(hdz.data, lighting=True, level=[0.4, 0.6, 0.9])

In [ ]:



# adapted from S.O.
def rotmatrix(axis,costheta):
    """ Calculate rotation matrix

    Arguments:
    - `axis`     : Rotation axis
    - `costheta` : Rotation angle
    """
    x,y,z = axis
    c = costheta
    s = sqrt(1-c*c)
    C = 1-c
    return  matrix([[ x*x*C+c,    x*y*C-z*s,  x*z*C+y*s ],
                    [ y*x*C+z*s,  y*y*C+c,    y*z*C-x*s ],
                    [ z*x*C-y*s,  z*y*C+x*s,  z*z*C+c   ]])

def plane(Lx,Ly,Nx,Ny,n,d):
    """ Calculate points of a generic plane 

    Arguments:
    - `Lx` : Plane Length first direction
    - `Ly` : Plane Length second direction
    - `Nx` : Number of points, first direction
    - `Ny` : Number of points, second direction
    - `n`  : Plane orientation, normal vector
    - `d`  : distance from the origin
    """

    x = linspace(-Lx/2,Lx/2,Nx)
    y = linspace(-Ly/2,Ly/2,Ny)
    # Create the mesh grid, of a XY plane sitting on the orgin
    X,Y = meshgrid(x,y)
    Z   = zeros([Nx,Ny])
    n0 = array([0,0,1])

    # Rotate plane to the given normal vector
    if any(n0!=n):
        costheta = dot(n0,n)/(norm(n0)*norm(n))
        axis     = cross(n0,n)/norm(cross(n0,n))
        rotMatrix = rotmatrix(axis,costheta)
        XYZ = vstack([X.flatten(),Y.flatten(),Z.flatten()])
        X,Y,Z = array(rotMatrix*XYZ).reshape(3,Nx,Ny)

    dVec = (n/norm(n))*d
    X,Y,Z = X+dVec[0],Y+dVec[1],Z+dVec[2]
    return X,Y,Z

In [ ]:
proc_img = get_window(wf1, 60, 0,0,vol_shape_x, vol_shape_y)
    
plt.imshow(proc_img)

In [ ]:
# prepare image 
proc_img_ubyte = img_as_ubyte(proc_img)
image = PIL.Image.fromarray(proc_img_ubyte)
proc_img_ubyte

In [ ]:


fig = p3.figure(figsize=(12,12))
p3.style.use('dark')


num_planes = 5
dist_list = linspace(0,165,num_planes)
print(dist_list)
norm_vec = array([0,0,1])





c1 =  np.array([0.6771093, 1.06297301, 0.54432319, 0.5])
X,Y,Z = plane(2000,2000,300,300,norm_vec,5)

X = X + 1000
Y = Y + 1000

#u = linspace(0,2000,100)
#v = linspace(0,2000,100)

u = np.array([k/X for k in range(2)])
v = np.array([k/Y for k in range(2)])

u = X / 2000 + 1000
v = Y / 2000 + 1000

mesh = p3.plot_mesh(X, Y, Z, u=u, v=v, texture=image, wireframe=False)

X,Y,Z = plane(2000,2000,100,100,norm_vec,120)
c2 =  np.array([0.2771093, 0.5, 0.54432319, 1.0])

X = X+1000
Y=Y+1000


#mesh2 = p3.plot_mesh(X, Y, Z, u=u, v=v, texture=image, wireframe=False)



#ipv.plot_surface(X, Y, Z,color=c2)

#X,Y,Z = plane(2000,2000,100,100,norm_vec,80)
#c3 =  np.array([0.771093, 0.97301, 0.54432319, 0.3])
#ipv.plot_surface(X, Z, Y, color=c3)


#p3.animation_control(mesh, interval=1000, sequence_length=4)

c3 =  np.array([0.0, 0.69, 0.65, 0.75])

#p3.volshow(, level=[0.25, 0.75], opacity=0.03, level_width=0.1, data_min=0, data_max=1)
scatter = p3.scatter(sel_clicks[:,1],  sel_clicks[:,2],  sel_clicks[:,0],color=c3,
                    size=1, marker='sphere')

p3.show()